Given a `.pkl.gz` output from periodogram search that has been visually confirmed as an EB (via the tornado webapp), I want to fit the EB signal and return a LC with that signal subtracted out (same `.pkl.gz` format, or at least with the same header info).

In [6]:
3.64159 * 2.101282

7.65200751838

In [2]:
ls nb-data/-data

ls: cannot access nb-data/-data: No such file or directory


# Now sketch

In [8]:
from astrobase import hatlc, checkplot, periodbase, plotbase
import numpy as np, matplotlib.pyplot as plt
import gzip
import pickle
import batman

detachedEBpath = 'nb-data/checkplot-'
hatid = 'HAT-199-0000452'
ptail = '.pkl.gz'

cpd = checkplot._read_checkplot_picklefile(detachedEBpath+hatid+ptail)
## Loading steps to get checkplot dictionary. N.b. this is _already normalized_ mags

detachedEBpath = '../data/detachedEBs/'
hatid = 'HAT-199-0000452'
ptail = '-V0-DR0-hatlc.sqlite.gz'
lcd, msg = hatlc.read_and_filter_sqlitecurve(detachedEBpath+hatid+ptail)
normlcd = hatlc.normalize_lcdict(lcd)                                                                                                                                                                
# Select recommended EPD aperture with 'G' flag. (Alternate                                                                                                                                          
# approach: take the smallest aperture to minimize crowding).                                                                                                                                        
# Get normalized unphased LC times, mags, errs
ap = next(iter(lcd['lcbestaperture']['ap']))                                                                                                                                                         
times = normlcd['rjd'][normlcd['aiq_'+ap]=='G']                                                                                                                                                      
mags = normlcd['aep_'+ap][normlcd['aiq_'+ap]=='G']                                                                                                                                                   
errs = normlcd['aie_'+ap][normlcd['aiq_'+ap]=='G']  

## checkplot dictionary should have varperiod and varepoch
cpd['varinfo']['varperiod'] = 2.101282
cpd['varinfo']['varepoch'] = 53956.94080

# Results from EB periodicity analysis:
varperiod = cpd['varinfo']['varperiod']
varepoch = cpd['varinfo']['varepoch']

# Initialize batman.
params = batman.TransitParams()
params.t0 = varepoch - 1.3            #time of inferior conjunction: not a multiple of main transit
params.per = varperiod*3.64159        #orbital period: ~3.2x that of EB (we want to find it)
params.rp = 0.02**(1/2.)             #planet radius: a 2% dip (units: stellar radii)
params.a = 9.                        #semi-major axis (in units of stellar radii)
params.inc = 89.                     #orbital inclination (in degrees)
params.ecc = 0.                      #eccentricity
params.w = 90.                       #longitude of periastron (in degrees)
params.u = [0.1, 0.3]                #limb darkening coefficients
params.limb_dark = "quadratic"       #limb darkening model

exp_time = 0.00388                   #rough exposure time (days). ~about 5.5minutes

m = batman.TransitModel(params,      #initialize model
                        times, 
                        supersample_factor = 7, 
                        exp_time = exp_time)

relflux = m.light_curve(params)       #calculates light curve
magdiff = -5/2. * np.log10(relflux)   #difference from median mag

# Inject planet, make plots
injmags = mags+magdiff
plt.close('all')
plt.scatter(times, mags)
plt.savefig('nb-data/plots/1_noinj-'+hatid+'.png', dpi=150)
plt.close('all')
plt.scatter(times, magdiff)
plt.savefig('nb-data/plots/0_transitmodel-'+hatid+'.png', dpi=150)
plt.close('all')
plt.scatter(times, injmags)
plt.savefig('nb-data/plots/2_injunphased-'+hatid+'.png', dpi=150)
plt.close('all')

# Periodicity analysis (with injected planet in data)
smallest_p = 0.5
biggest_p = min((times[-1] - times[0])/2.01, 100.)

print('\nStellingwerf...\n')
spdmp = periodbase.stellingwerf_pdm(times,injmags,errs,
                    autofreq=True,
                    startp=smallest_p,
                    endp=biggest_p,
                    normalize=False,
                    stepsize=1.0e-5,
                    phasebinsize=0.03,
                    mindetperbin=9,
                    nbestpeaks=5,
                    periodepsilon=0.1, # 0.1days
                    sigclip=None, # no sigma clipping
                    nworkers=None)

varinfo = {'objectisvar':True,
          'vartags':None,
          'varisperiodic':True,
          'varperiod':spdmp['bestperiod'],
          'varepoch':None}

print('\nBLS...\n')
blsp = periodbase.bls_parallel_pfind(times, injmags, errs,
                                    startp=smallest_p,
                                    endp=biggest_p,
                                    stepsize=1.0e-5,
                                    mintransitduration=0.01,
                                    maxtransitduration=0.6,
                                    nphasebins=200,
                                    autofreq=False,
                                    nbestpeaks=5,
                                    periodepsilon=0.1,
                                    nworkers=None,
                                    sigclip=None)

lspinfolist = [spdmp, blsp]
cp = checkplot.checkplot_pickle(lspinfolist, times, injmags, errs,
                               nperiodstouse=3,
                               objectinfo=normlcd['objectinfo'],
                               varinfo=varinfo,
                               findercmap='gray_r',
                               normto='globalmedian',
                               normmingap=4.,
                               outfile='nb-data/checkplot-inj-'+hatid+'.pkl.gz',
                               sigclip=[10.,-3.],
                               varepoch='min',
                               phasewrap=True,
                               phasesort=True,
                               phasebin=0.002,
                               plotxlim=[-0.6,0.6],
                               plotdpi=120,
                               returndict=False,
                               pickleprotocol=3,
                               greenhighlight=False,
                               xgridlines=[-0.5,0.,0.5])

#since we don't have easy checkplot_pickle to png written yet, use twolsp_checkplot_png
injpath = 'nb-data/plots/3_checkplot-inj-'

checkplot.twolsp_checkplot_png(spdmp, blsp, times, injmags, errs,
                               objectinfo=normlcd['objectinfo'],
                               findercmap='gray_r',
                               normto='globalmedian',
                               normmingap=4.,
                               outfile=injpath+hatid+'.png',
                               sigclip=[10.,-3.],
                               varepoch='min',
                               phasewrap=True,
                               phasesort=True,
                               phasebin=0.002,
                               plotxlim=[-0.6,0.6],
                               plotdpi=150)
plt.close('all')


#
#Now subtract median EB signal
#
from scipy.interpolate import interp1d

#confirm EB period.
injEBpath = 'nb-data/checkplot-inj-'
hatid = 'HAT-199-0000452'
ptail = '.pkl.gz'
cpd = checkplot._read_checkplot_picklefile(injEBpath+hatid+ptail)

#unphased LC times, mags, errs
times = cpd['magseries']['times']
mags = cpd['magseries']['mags']
errs = cpd['magseries']['errs']
#phased data, then binned data (blue points)
phase, phasedmags = cpd['pdm'][0]['phase'], cpd['pdm'][0]['phasedmags']
binphase, binphasedmags = cpd['pdm'][0]['binphase'], cpd['pdm'][0]['binphasedmags']

twophasetimes = varepoch + (binphase*varperiod) #times over two phases (stored in binphase originally)

#now extend twophasetimes to beginning and end of baseline
modeltimes = np.array(twophasetimes)
modelmags = binphasedmags

exitf,upf,downf = False,True,True
diff = modeltimes[-1] - modeltimes[0]

while not exitf:
    
    while upf:
        onetimeup = np.array(modeltimes[-1] - modeltimes[0] + twophasetimes)[1:]                
        onemagup = np.array(binphasedmags[1:])
        #oneerrup = np.array() #obnoxiously, these aren't kept thru the phase curve-making. should be needed to propagate uncertainties
        
        modeltimes = np.append(modeltimes, onetimeup)
        modelmags = np.append(modelmags, onemagup)
        if max(modeltimes) > max(times):
            upf = False
    
    k = 1
    while downf:
            #careful of lengths
            onetimedown = np.array(-k*diff + twophasetimes)[:-1]
            onemagdown = np.array(binphasedmags[:-1])
            #oneerrup = np.array() #not kept thru phase curve making 

            modeltimes = np.insert(modeltimes, 0, onetimedown)
            modelmags = np.insert(modelmags, 0, onemagdown)

            k+=1

            if min(modeltimes) < min(times):
                downf = False
                exitf = True

f = interp1d(modeltimes, modelmags, kind='linear')
                
interpmodelmags = f(times)

residualmags = mags - interpmodelmags # n.b. these mags are the injected ones

#save figure showing model and residuals
plt.close('all')
f, axs = plt.subplots(3, sharex=True, figsize=(15,8*1.5))
axs[0].scatter(times, mags, lw=0, c='black', s=10)
axs[0].set(ylabel='DEB data (w/ planet injected)')
axs[1].scatter(times, interpmodelmags, lw=0, c='black', s=10)
axs[1].set(ylabel='median model\n(medians from binned phase-folded DEB LC)')
axs[2].scatter(times, residualmags, lw=0, c='black', s=10)
axs[2].set(ylabel='data - model')
f.tight_layout()
f.savefig('nb-data/plots/4_median_EB_modelLC_and_residuals.png', dpi=150)
plt.close('all')
print('\nMade figure of median EB model and residuals...\n')


#
# NOW RERUN PERIODICITY ANALYSIS ON THE RESIDUALS
#

# Periodicity analysis (with injected planet in data)

times = times
mags = residualmags
errs = errs #TODO: propagate model uncertainty to residual magnitudes

smallest_p = varperiod*2. + 0.1
biggest_p = min((times[-1] - times[0])/2.01, 100.)

print('\nStellingwerf...\n')
spdmp = periodbase.stellingwerf_pdm(times,mags,errs,
                    autofreq=True,
                    startp=smallest_p,
                    endp=biggest_p,
                    normalize=False,
                    stepsize=5.0e-5,
                    phasebinsize=0.05,
                    mindetperbin=9,
                    nbestpeaks=5,
                    periodepsilon=0.1, # 0.1days
                    sigclip=None, # no sigma clipping
                    nworkers=None)

print('\nBLS...\n')
blsp = periodbase.bls_parallel_pfind(times, mags, errs,
                                    startp=smallest_p,
                                    endp=biggest_p,
                                    stepsize=5.0e-5,
                                    mintransitduration=0.01,
                                    maxtransitduration=0.6,
                                    nphasebins=200,
                                    autofreq=False,
                                    nbestpeaks=5,
                                    periodepsilon=0.1,
                                    nworkers=None,
                                    sigclip=None)

lspinfolist = [spdmp, blsp]
cp = checkplot.checkplot_pickle(lspinfolist, times, mags, errs,
                               nperiodstouse=3,
                               objectinfo=normlcd['objectinfo'],
                               varinfo=varinfo,
                               findercmap='gray_r',
                               normto='globalmedian',
                               normmingap=4.,
                               outfile='nb-data/checkplot-injresiduals-'+hatid+'.pkl.gz',
                               sigclip=[10.,-3.],
                               varepoch='min',
                               phasewrap=True,
                               phasesort=True,
                               phasebin=0.002,
                               plotxlim=[-0.6,0.6],
                               plotdpi=120,
                               returndict=False,
                               pickleprotocol=3,
                               greenhighlight=False,
                               xgridlines=[-0.5,0.,0.5])

#since we don't have easy checkplot_pickle to png written yet, use twolsp_checkplot_png
injpath = 'nb-data/plots/5_checkplot-injresiduals-'

checkplot.twolsp_checkplot_png(spdmp, blsp, times, mags, errs,
                               objectinfo=normlcd['objectinfo'],
                               findercmap='gray_r',
                               normto='globalmedian',
                               normmingap=4.,
                               outfile=injpath+hatid+'.png',
                               sigclip=[10.,-3.],
                               varepoch='min',
                               phasewrap=True,
                               phasesort=True,
                               phasebin=0.002,
                               plotxlim=[-0.6,0.6],
                               plotdpi=150)
plt.close('all')

print('done.')


2017-01-23T16:55:26.371991Z [INFO]: retrieving all latest columns
2017-01-23T16:55:26.372197Z [INFO]: no LC filters specified
2017-01-23T16:55:26.641087Z [WRN!]: no sdssr available in lcdict, normalizing to global mag median
2017-01-23T16:55:26.661544Z [WRN!]: column atf_000 is all nan, skipping...
2017-01-23T16:55:26.662998Z [WRN!]: column atf_001 is all nan, skipping...

Stellingwerf...

2017-01-23T16:55:28.220249Z [INFO]: using autofreq with 2046 frequency points, start P = 0.500, end P = 100.000
2017-01-23T16:55:28.220287Z [INFO]: using 16 workers...

BLS...

2017-01-23T16:55:29.263335Z [INFO]: manualfreq: using stepsize: 1e-05, min P: 0.5, max P: 100.0, nfreq: 199000, nphasebins: 200, min transit duration: 0.01, max transit duration: 0.6
2017-01-23T16:55:29.263376Z [INFO]: manualfreq: minfreq: 0.01, maxfreq: 2.0
2017-01-23T16:55:29.263433Z [INFO]: using 16 workers...
2017-01-23T16:55:29.265032Z [INFO]: worker 1: minfreq = 0.010, nfreqs = 12438
2017-01-23T16:55:29.265063Z [INFO]: w

/home/luke/Dropbox/miniconda3/envs/sci/lib/python3.5/site-packages/astroquery/utils/commons.py:187: UserWarning: Coordinate string is being interpreted as an ICRS coordinate.
  warnings.warn("Coordinate string is being interpreted as an "


2017-01-23T16:55:36.104996Z [INFO]: sigclip = [10.0, -3.0]: before = 4351 observations, after = 4293 observations
2017-01-23T16:55:37.587028Z [INFO]: sigclip = 30.0: before = 4293 observations, after = 4293 observations
2017-01-23T16:55:37.589682Z [INFO]: spline fit done. nknots = 42,  chisq = 264455.09709, reduced chisq = 62.22473
2017-01-23T16:55:37.589763Z [INFO]: plotting pdm phased LC with period 0: 2.101152, epoch: 54055.69671
2017-01-23T16:55:38.272501Z [INFO]: sigclip = 30.0: before = 4293 observations, after = 4293 observations
2017-01-23T16:55:38.275096Z [INFO]: spline fit done. nknots = 42,  chisq = 535915.08167, reduced chisq = 126.09767
2017-01-23T16:55:38.275177Z [INFO]: plotting pdm phased LC with period 1: 1.050877, epoch: 54035.76238
2017-01-23T16:55:38.956574Z [INFO]: sigclip = 30.0: before = 4293 observations, after = 4293 observations
2017-01-23T16:55:38.959113Z [INFO]: spline fit done. nknots = 42,  chisq = 524965.58830, reduced chisq = 123.52131
2017-01-23T16:55:3

/home/luke/Dropbox/miniconda3/envs/sci/lib/python3.5/site-packages/matplotlib/figure.py:1744: UserWarning: This figure includes Axes that are not compatible with tight_layout, so its results might be incorrect.
  warnings.warn("This figure includes Axes that are not "


2017-01-23T16:55:54.057962Z [INFO]: checkplot done -> nb-data/plots/3_checkplot-inj-HAT-199-0000452.png

Made figure of median EB model and residuals...


Stellingwerf...

2017-01-23T16:55:55.743003Z [INFO]: using autofreq with 229 frequency points, start P = 4.314, end P = 100.000
2017-01-23T16:55:55.743042Z [INFO]: using 16 workers...

BLS...

2017-01-23T16:55:56.119247Z [INFO]: manualfreq: using stepsize: 5e-05, min P: 4.302563999999999, max P: 100.0, nfreq: 4449, nphasebins: 200, min transit duration: 0.01, max transit duration: 0.6
2017-01-23T16:55:56.119303Z [INFO]: manualfreq: minfreq: 0.01, maxfreq: 0.23241955262025157
2017-01-23T16:55:56.119363Z [INFO]: using 16 workers...
2017-01-23T16:55:56.119540Z [INFO]: worker 1: minfreq = 0.010, nfreqs = 279
2017-01-23T16:55:56.119570Z [INFO]: worker 2: minfreq = 0.024, nfreqs = 279
2017-01-23T16:55:56.119590Z [INFO]: worker 3: minfreq = 0.038, nfreqs = 279
2017-01-23T16:55:56.119609Z [INFO]: worker 4: minfreq = 0.052, nfreqs = 279
2017-

# Now see if the planet is still there